#### 1. 准备红头文件匹配项

#### 2. 处理文件所需要的共有方法

In [2]:
def get_stop_words():
    '''停用词表'''
    with open('data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    '''分词'''
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])

def handle_desc(desc):
    '''对企业描述分词'''
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']

def getLDA(df,topic_num):
    from gensim.models import LdaMulticore
    from gensim.corpora import Dictionary
    # gensim
    dictionary = Dictionary(df["经营业务范围"])
    print(dictionary)
    corpus = [dictionary.doc2bow(doc) for doc in df["经营业务范围"]]
    lda = LdaMulticore(corpus=corpus,  # LDA训练语料
                   id2word=dictionary, # id到单词的映射表
                   num_topics=topic_num)      # LDA主题数量 
    return lda,corpus,dictionary

def get_topic_distribute(lda,doc):
    return lda[doc]

def get_all_doc_topic(df):
    lda_matrix = np.zeros((df["经营业务范围"].shape[0],topic_num),dtype='float64')
    for index,doc in enumerate(corpus):
        topics = get_topic_distribute(lda,corpus[index])
        for topic_tuple in topics:
            lda_matrix[index,topic_tuple[0]] = topic_tuple[1]
    return lda_matrix

#### 分词输入文件

In [3]:
import jieba
import numpy as np

In [4]:
inputdf = pd.read_csv("newdata/input.csv",index_col=0)

In [5]:
inputdf["经营业务范围"] = inputdf["经营业务范围"].apply(cut)
inputdf["经营业务范围"] = inputdf["经营业务范围"].apply(lambda x:x.split("/"))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
inputdf["label"] = le.fit_transform(inputdf["label"])
print(le.classes_)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.605 seconds.
Prefix dict has been built succesfully.


['信息产业' '医养健康' '文化产业' '旅游产业' '旧动能' '海洋经济' '现代农业' '现代金融' '能源原材料' '能源新材料'
 '高端化工' '高端装备制造']


In [6]:
inputdf.head()

,label,企业名称,所在城市,经营业务范围,统一信用代码
0,11,济南二机床集团有限公司技术中心,山东省,"[道路, 货物运输, 普通, 货运, 三类, 汽车, 维修, 车身, 清洁, 维护, 第一类...",-
1,0,浪潮集团有限公司技术中心,山东省,"[商用, 密码, 产品, 开发, 生产, 销售, 有效期限, 许可证, 为准, 计算机, 软...",-
2,1,鲁南制药集团股份有限公司技术中心,山东省,"[生产, 销售, 原料药, 包装, 物品, 依法, 须, 批准, 项目, 相关, 部门, 批...",-
3,11,济南轻骑摩托车有限公司技术中心,山东省,"[摩托车, 零配件, 设计, 开发, 生产, 销售, 摩托车, 技术咨询, 服务, 引进, ...",-
4,10,万华化学集团股份有限公司技术中心,山东省,"[安全, 生产, 许可证, 范围, 化学, 危险品, 生产, 许可证, 范围, 铁路, 专用...",-


In [7]:
topic_num = 25
lda,corpus,dictionary = getLDA(inputdf,topic_num)
#lda.print_topics()
print(get_topic_distribute(lda,corpus[0]))
print(len(corpus))
# 所有样本主题分布
all_topic = get_all_doc_topic(inputdf)

Dictionary(3107 unique tokens: ['GC2', '三类', '专用设备', '业务', '为准']...)
[(0, 0.028253261), (15, 0.22203906), (21, 0.16987671), (22, 0.5739977)]
1320


In [8]:
all_topic

array([[0.03582705, 0.        , 0.        , ..., 0.57347691, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.29690465, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01333333, 0.01333333, 0.01333333, ..., 0.01333333, 0.01333333,
        0.68000001]])

#### 分类

In [9]:
x = all_topic
y = pd.get_dummies(inputdf["label"])
print(x.shape,y.shape)

(1320, 25) (1320, 12)


In [12]:
x[0]

array([0.03582705, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.21375732, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.17110544, 0.57347691, 0.        , 0.        ])

In [16]:
import keras 
from keras import models
from keras import layers
network = models.Sequential()
# input_shape : 输入张量的形状, (28*28,)表示1维度向量
network.add(layers.Dense(32,activation='relu',input_shape=(x.shape[1],)))
network.add(layers.Dense(len(le.classes_),activation='softmax'))
# 4. 编译
network.compile(optimizer = 'rmsprop',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

# 5. 训练模型
network.fit(x,y,epochs=60,batch_size=2)

Epoch 1/60
1320/1320 [==============================] - 0s 377us/step - loss: 0.9995 - acc: 0.8439
Epoch 2/60
1320/1320 [==============================] - 0s 307us/step - loss: 0.6097 - acc: 0.8765
Epoch 3/60
1320/1320 [==============================] - 0s 330us/step - loss: 0.6065 - acc: 0.8765
Epoch 4/60
1320/1320 [==============================] - 0s 330us/step - loss: 0.6250 - acc: 0.8765
Epoch 5/60
1320/1320 [==============================] - 0s 355us/step - loss: 0.6324 - acc: 0.8765
Epoch 6/60
1320/1320 [==============================] - 0s 332us/step - loss: 0.6584 - acc: 0.8765
Epoch 7/60
1320/1320 [==============================] - 0s 311us/step - loss: 0.6592 - acc: 0.8765
Epoch 8/60
1320/1320 [==============================] - 0s 307us/step - loss: 0.6728 - acc: 0.8765
Epoch 9/60
1320/1320 [==============================] - 0s 303us/step - loss: 0.7058 - acc: 0.8765
Epoch 10/60
1320/1320 [==============================] - 0s 310us/step - loss: 0.6962 - acc: 0.8765
Epoch 11/

#### 保存

In [ ]:
import pickle
#保存词典
with open('newdata/dict_save', 'wb') as f:                     
    picklestring = pickle.dump(dictionary, f)
#保存lda模型
lda.save('newdata/lda.model')
network.save('newdata/my_model.h5') # 保存模型
network.save_weights('newdata/my_model_weights.h5') # 保存参数

### 预测 - tableau调用

In [33]:
import pickle
import pandas as pd
import numpy as np
import jieba
from keras.models import load_model

def get_stop_words():
    with open('data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']

[(0, 0.04), (1, 0.04), (2, 0.04), (3, 0.04), (4, 0.04), (5, 0.04), (6, 0.04), (7, 0.04), (8, 0.04), (9, 0.04), (10, 0.04), (11, 0.04), (12, 0.04), (13, 0.04), (14, 0.04), (15, 0.04), (16, 0.04), (17, 0.04), (18, 0.04), (19, 0.04), (20, 0.04), (21, 0.04), (22, 0.04), (23, 0.04), (24, 0.04)]
(0, 0.04)
(1, 0.04)
(2, 0.04)
(3, 0.04)
(4, 0.04)
(5, 0.04)
(6, 0.04)
(7, 0.04)
(8, 0.04)
(9, 0.04)
(10, 0.04)
(11, 0.04)
(12, 0.04)
(13, 0.04)
(14, 0.04)
(15, 0.04)
(16, 0.04)
(17, 0.04)
(18, 0.04)
(19, 0.04)
(20, 0.04)
(21, 0.04)
(22, 0.04)
(23, 0.04)
(24, 0.04)
[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04]
<class 'numpy.ndarray'>


In [65]:
import pandas as pd
def getredItem(city):
    df = pd.read_csv("newdata/redhead.csv",delimiter="$",header=None)
    df.columns = ["地市","红头文件描述"]
    res = df[df["地市"]==city]["红头文件描述"].tolist()
    print(res)
    return res[0]
getredItem("山东省")

def getResult(corpName,topic_num):
    res_dict = {}
    with open('newdata/dict_save', 'rb') as f:
        load_dictionary = pickle.load(f)   # read file and build object
    from gensim import models
    load_lda = models.ldamodel.LdaModel.load('newdata/lda.model')

    model = load_model('newdata/my_model.h5')
    model.load_weights('newdata/my_model_weights.h5')
    
    alldf = pd.read_csv("newdata/input.csv",index_col=0)
    alldf = alldf[alldf["企业名称"]==corpName]
    desc = alldf["经营业务范围"][0]
    
    a = load_dictionary.doc2bow(handle_desc(desc))
    topic_distribute = load_lda[a]
#     print(topic_distribute)
    x_test = np.zeros((topic_num,))
    for tupl in topic_distribute:
#         print(tupl)
        x_test[tupl[0]]=tupl[1]
    res = model.predict(x_test.reshape(1,topic_num))
    res_dict["label"] = le.inverse_transform([np.argmax(res[0])])[0]
    res_dict["经营范围"] = desc
    res_dict["所属城市"] = alldf["所在城市"][0]
    print(alldf["所在城市"][0])
    res_dict["公司名称"] = corpName
    res_dict["红头文件描述项目"] = getredItem(alldf["所在城市"][0])
    
    return res_dict
    
topic_num = 25
corpName = "济南二机床集团有限公司技术中心"
getResult(corpName,topic_num)

山东省
['红头文件可修改..']


/devkits/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'label': '旧动能',
 '经营范围': '道路货物运输(普通货运);三类汽车维修(车身清洁维护);第一类压力容器、第二类低中压容器的设计、制造、销售,工业氧气(以上经营项目有效期限以许可证为准);金属成型机床设备、金属切削机床设备、工业自动控制系统装置、铸造机械设备、金属切割及焊接设备、建材及环境保护专用设备、机床附件、配电开关控制设备、塑料工业设备、包装机械设备、模具的设计、制造、销售;机床设备安装调试、维修改造;A级起重机械安装、GC2压力管道安装(以上凭资质证经营);机械零部件加工;金属材料、铸件、五金交电、橡胶制品、木制品、工夹量仪、汽车、汽车配件的销售;进出口业务;装卸服务;机械设备技术开发;企业管理咨询;计算机信息技术服务;软件开发;检测技术服务;经济贸易代理;职业技能鉴定;房屋租赁;场地租赁;物业管理服务;会议会展服务;分支机构经营:金属材料的焊接、锻造和热处理;提供培训场地、住宿,餐饮,酒水、卷烟的零售(以上凭许可证经营)。(依法须经批准的项目,经相关部门批准后方可开展经营活动)。',
 '所属城市': '山东省',
 '公司名称': '济南二机床集团有限公司技术中心',
 '红头文件描述项目': '红头文件可修改..'}

### 附录: 整理爬去的数据为输入格式

In [ ]:
input1 = pd.read_csv("newdata/qichacha.csv",index_col=0)
input1 = input1[["公司名","统一社会信用代码","所属地区","经营范围"]]
input1.columns = ["企业名称","统一信用代码","所在城市","经营业务范围"]
input1["label"] = "旧动能"
input1.head()
pos_df = pd.read_excel("pos.xlsx")
pos_df = pd.DataFrame({"公司名称":pos_df["公司名称"],
                       "经营范围":pos_df["经营范围"],
                       "企业分类":pos_df["企业分类"]})
pos_df = pos_df.dropna(how="any",axis=0)
pos_df.columns = ["企业名称","经营业务范围","label"]
pos_df["统一信用代码"] = "-"
pos_df["所在城市"] = "山东省"
pos_df.head()
alldf = pd.concat([pos_df,input1],axis=0)
alldf.to_csv("newdata/input.csv")